In [148]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import fileinput, glob, os
import datetime as dt
import missingno as msno
import seaborn as sns

In [149]:
#Import from stock_list.csv, filter by Universe=True and parse columns with dates:
stock_list = pd.read_csv('files/stock_list.csv', index_col=0, parse_dates=[1])
stock_list_incl = stock_list[stock_list.Universe0 == True]
stock_list_incl.TradeDate = pd.to_datetime(stock_list_incl.TradeDate, format='%Y%m%d')
stock_list_incl.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2000 entries, 1301 to 9997
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   EffectiveDate           2000 non-null   datetime64[ns]
 1   Name                    2000 non-null   object        
 2   Section/Products        2000 non-null   object        
 3   NewMarketSegment        2000 non-null   object        
 4   33SectorCode            2000 non-null   object        
 5   33SectorName            2000 non-null   object        
 6   17SectorCode            2000 non-null   object        
 7   17SectorName            2000 non-null   object        
 8   NewIndexSeriesSizeCode  2000 non-null   object        
 9   NewIndexSeriesSize      2000 non-null   object        
 10  TradeDate               2000 non-null   datetime64[ns]
 11  Close                   2000 non-null   float64       
 12  IssuedShares            2000 non-null   float

/Users/koytoora/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [150]:
stock_list_incl.head()

,EffectiveDate,Name,Section/Products,NewMarketSegment,33SectorCode,33SectorName,17SectorCode,17SectorName,NewIndexSeriesSizeCode,NewIndexSeriesSize,TradeDate,Close,IssuedShares,MarketCapitalization,Universe0
SecuritiesCode,,,,,,,,,,,,,,,
1301,2021-12-30,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,50,"Fishery, Agriculture and Forestry",1,FOODS,7,TOPIX Small 2,2021-12-30,3080.0,10928283.0,3.365911e+10,True
1332,2021-12-30,"Nippon Suisan Kaisha,Ltd.",First Section (Domestic),Prime Market,50,"Fishery, Agriculture and Forestry",1,FOODS,4,TOPIX Mid400,2021-12-30,543.0,312430277.0,1.696496e+11,True
1333,2021-12-30,Maruha Nichiro Corporation,First Section (Domestic),Prime Market,50,"Fishery, Agriculture and Forestry",1,FOODS,4,TOPIX Mid400,2021-12-30,2405.0,52656910.0,1.266399e+11,True
1375,2021-12-30,"YUKIGUNI MAITAKE CO.,LTD.",First Section (Domestic),Prime Market,50,"Fishery, Agriculture and Forestry",1,FOODS,6,TOPIX Small 1,2021-12-30,1196.0,39910700.0,4.773320e+10,True
1376,2021-12-30,"KANEKO SEEDS CO.,LTD.",First Section (Domestic),Standard Market,50,"Fishery, Agriculture and Forestry",1,FOODS,7,TOPIX Small 2,2021-12-30,1504.0,11772626.0,1.770603e+10,True


In [151]:
#Import from stock_prices.csv and parse columns with dates:
stock_prices = pd.read_csv('files/train_files/stock_prices.csv', index_col=0, parse_dates=[1])

In [166]:
stock_prices.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2332531 entries, 20170104_1301 to 20211203_9997
Data columns (total 11 columns):
 #   Column            Dtype         
---  ------            -----         
 0   Date              datetime64[ns]
 1   SecuritiesCode    int64         
 2   Open              float64       
 3   High              float64       
 4   Low               float64       
 5   Close             float64       
 6   Volume            int64         
 7   AdjustmentFactor  float64       
 8   ExpectedDividend  float64       
 9   SupervisionFlag   bool          
 10  Target            float64       
dtypes: bool(1), datetime64[ns](1), float64(7), int64(2)
memory usage: 198.0+ MB


In [385]:
stock_prices.head()

,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target
RowId,,,,,,,,,,,
20170104_1301,2017-01-04,1301,2734.0,2755.0,2730.0,2742.0,31400,1.0,NaN,False,0.000730
20170104_1332,2017-01-04,1332,568.0,576.0,563.0,571.0,2798500,1.0,NaN,False,0.012324
20170104_1333,2017-01-04,1333,3150.0,3210.0,3140.0,3210.0,270800,1.0,NaN,False,0.006154
20170104_1376,2017-01-04,1376,1510.0,1550.0,1510.0,1550.0,11300,1.0,NaN,False,0.011053
20170104_1377,2017-01-04,1377,3270.0,3350.0,3270.0,3330.0,150800,1.0,NaN,False,0.003026


In [153]:
#Import from options.csv and parse columns with dates:
options = pd.read_csv('files/train_files/options.csv', index_col=0, parse_dates=[1],
                      converters={7:str,8:str,9:str,10:str})

In [154]:
#Correction of data types in some columns
options.LastTradingDay = pd.to_datetime(options.LastTradingDay, format='%Y%m%d')
options.SpecialQuotationDay = pd.to_datetime(options.SpecialQuotationDay, format='%Y%m%d')
options.NightSessionOpen = options.NightSessionOpen.str.replace('-','0')
options.NightSessionOpen = options.NightSessionOpen.astype('float')
options.NightSessionHigh = options.NightSessionHigh.str.replace('-','0')
options.NightSessionHigh = options.NightSessionHigh.astype('float')
options.NightSessionLow = options.NightSessionLow.str.replace('-','0')
options.NightSessionLow = options.NightSessionLow.astype('float')
options.NightSessionClose = options.NightSessionClose.str.replace('-','0')
options.NightSessionClose = options.NightSessionClose.astype('float')

In [155]:
options.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3567694 entries, 20170104_132010018 to 20211203_199248018
Data columns (total 30 columns):
 #   Column               Dtype         
---  ------               -----         
 0   Date                 datetime64[ns]
 1   OptionsCode          int64         
 2   WholeDayOpen         float64       
 3   WholeDayHigh         float64       
 4   WholeDayLow          float64       
 5   WholeDayClose        float64       
 6   NightSessionOpen     float64       
 7   NightSessionHigh     float64       
 8   NightSessionLow      float64       
 9   NightSessionClose    float64       
 10  DaySessionOpen       float64       
 11  DaySessionHigh       float64       
 12  DaySessionLow        float64       
 13  DaySessionClose      float64       
 14  TradingVolume        int64         
 15  OpenInterest         int64         
 16  TradingValue         int64         
 17  ContractMonth        int64         
 18  StrikePrice          float64       
 19

In [156]:
#Import from trades.csv, drop NaNs, and parse columns with dates:
trades = pd.read_csv('files/train_files/trades.csv', parse_dates=[0,1,2])
trades_full = trades.dropna()

In [157]:
trades_full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 765 entries, 6 to 1710
Data columns (total 56 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   Date                                 765 non-null    datetime64[ns]
 1   StartDate                            765 non-null    datetime64[ns]
 2   EndDate                              765 non-null    datetime64[ns]
 3   Section                              765 non-null    object        
 4   TotalSales                           765 non-null    float64       
 5   TotalPurchases                       765 non-null    float64       
 6   TotalTotal                           765 non-null    float64       
 7   TotalBalance                         765 non-null    float64       
 8   ProprietarySales                     765 non-null    float64       
 9   ProprietaryPurchases                 765 non-null    float64       
 10  ProprietaryTo

In [158]:
#Import from financials.csv and parse dates:
financials = pd.read_csv('files/train_files/financials.csv', index_col=0, parse_dates=[2,4,8,10,11],
                        converters={14:str,15:str,16:str,17:str,18:str,19:str,42:str,44:str})

In [159]:
financials.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 92956 entries, 20161207453651.0 to 20211203446277.0
Data columns (total 44 columns):
 #   Column                                                                        Non-Null Count  Dtype         
---  ------                                                                        --------------  -----         
 0   DateCode                                                                      92954 non-null  object        
 1   Date                                                                          92956 non-null  datetime64[ns]
 2   SecuritiesCode                                                                92954 non-null  float64       
 3   DisclosedDate                                                                 92954 non-null  datetime64[ns]
 4   DisclosedTime                                                                 92954 non-null  object        
 5   DisclosedUnixTime                                           

In [367]:
#Это бесполезные операции, сначала генерирурю список дней и акций, а потом джойном удаляю строки, для которых нет данных в stock_prices
#Просто потренировался, можно сразу брать индексы из stock_prices
date_range = pd.date_range(stock_prices.Date.min(), stock_prices.Date.max(), freq='D')
dates = np.array(date_range)
stocks = np.array(stock_list_incl.index)
multi_index = pd.MultiIndex.from_product([dates, stocks], names=['Date','SecuritiesCode'])
final_full = multi_index.to_frame(index = False)
final = final_full.merge(stock_prices.iloc[:,:2], how='inner', on=['Date','SecuritiesCode'])
final

,Date,SecuritiesCode
0,2017-01-04,1301
1,2017-01-04,1332
2,2017-01-04,1333
3,2017-01-04,1376
4,2017-01-04,1377
...,...,...
2332526,2021-12-03,9990
2332527,2021-12-03,9991
2332528,2021-12-03,9993
2332529,2021-12-03,9994


In [383]:
#Скользящее среднее за 50 дней
sma50 = stock_prices.set_index('Date').groupby(['SecuritiesCode'])['Close'].rolling(window='50D').mean()
final2 = final.merge(sma50, how='left', left_on=['SecuritiesCode','Date'], right_index=True)
final2 = final2.rename(columns={'Close': 'SMA50'})
final2

,Date,SecuritiesCode,SMA50
0,2017-01-04,1301,2742.000000
1,2017-01-04,1332,571.000000
2,2017-01-04,1333,3210.000000
3,2017-01-04,1376,1550.000000
4,2017-01-04,1377,3330.000000
...,...,...,...
2332526,2021-12-03,9990,563.794118
2332527,2021-12-03,9991,831.617647
2332528,2021-12-03,9993,1708.117647
2332529,2021-12-03,9994,2404.852941


In [384]:
#Скользящее среднее за 10 дней
sma10 = stock_prices.set_index('Date').groupby(['SecuritiesCode'])['Close'].rolling(window='10D').mean()
final3 = final2.merge(sma10, how='left', left_on=['SecuritiesCode','Date'], right_index=True)
final3 = final3.rename(columns={'Close': 'SMA10'})
final3

,Date,SecuritiesCode,SMA50,SMA10
0,2017-01-04,1301,2742.000000,2742.000
1,2017-01-04,1332,571.000000,571.000
2,2017-01-04,1333,3210.000000,3210.000
3,2017-01-04,1376,1550.000000,1550.000
4,2017-01-04,1377,3330.000000,3330.000
...,...,...,...,...
2332526,2021-12-03,9990,563.794118,532.500
2332527,2021-12-03,9991,831.617647,787.875
2332528,2021-12-03,9993,1708.117647,1680.625
2332529,2021-12-03,9994,2404.852941,2370.125


In [394]:
#Цена закрытия из stock_prices
final4 = final3.merge(stock_prices.iloc[:,[0, 1, 5]], how='left', on=['Date','SecuritiesCode'])
final4

,Date,SecuritiesCode,SMA50,SMA10,Close
0,2017-01-04,1301,2742.000000,2742.000,2742.0
1,2017-01-04,1332,571.000000,571.000,571.0
2,2017-01-04,1333,3210.000000,3210.000,3210.0
3,2017-01-04,1376,1550.000000,1550.000,1550.0
4,2017-01-04,1377,3330.000000,3330.000,3330.0
...,...,...,...,...,...
2332526,2021-12-03,9990,563.794118,532.500,528.0
2332527,2021-12-03,9991,831.617647,787.875,794.0
2332528,2021-12-03,9993,1708.117647,1680.625,1645.0
2332529,2021-12-03,9994,2404.852941,2370.125,2389.0


In [395]:
#Относительная разница цен закрытия и скользящих средних
final4['SMA50_Close_diff'] = (final4.Close-final4.SMA50) / final4.Close
final4['SMA10_Close_diff'] = (final4.Close-final4.SMA10) / final4.Close
final4

,Date,SecuritiesCode,SMA50,SMA10,Close,SMA50_Close_diff,SMA10_Close_diff
0,2017-01-04,1301,2742.000000,2742.000,2742.0,0.000000,0.000000
1,2017-01-04,1332,571.000000,571.000,571.0,0.000000,0.000000
2,2017-01-04,1333,3210.000000,3210.000,3210.0,0.000000,0.000000
3,2017-01-04,1376,1550.000000,1550.000,1550.0,0.000000,0.000000
4,2017-01-04,1377,3330.000000,3330.000,3330.0,0.000000,0.000000
...,...,...,...,...,...,...,...
2332526,2021-12-03,9990,563.794118,532.500,528.0,-0.067792,-0.008523
2332527,2021-12-03,9991,831.617647,787.875,794.0,-0.047377,0.007714
2332528,2021-12-03,9993,1708.117647,1680.625,1645.0,-0.038369,-0.021657
2332529,2021-12-03,9994,2404.852941,2370.125,2389.0,-0.006636,0.007901


In [402]:
# Доля объемов на акцию из общедневного объема торговли
temp1 = stock_prices.groupby('Date').agg('sum')['Volume']
final5 = final4.merge(stock_prices.iloc[:,[0, 1, 6]], how='left', on=['Date','SecuritiesCode'])
final5 = final5.rename(columns={'Volume': 'ind_volume'})
final6 = final5.merge(temp1, how='left', left_on='Date', right_index=True)
final6 = final6.rename(columns={'Volume': 'tot_volume'})
final6['Volume_share'] = final6['ind_volume'] / final6['tot_volume']
final6

,Date,SecuritiesCode,SMA50,SMA10,Close,SMA50_Close_diff,SMA10_Close_diff,ind_volume,tot_volume,Volume_share
0,2017-01-04,1301,2742.000000,2742.000,2742.0,0.000000,0.000000,31400,2334312600,0.000013
1,2017-01-04,1332,571.000000,571.000,571.0,0.000000,0.000000,2798500,2334312600,0.001199
2,2017-01-04,1333,3210.000000,3210.000,3210.0,0.000000,0.000000,270800,2334312600,0.000116
3,2017-01-04,1376,1550.000000,1550.000,1550.0,0.000000,0.000000,11300,2334312600,0.000005
4,2017-01-04,1377,3330.000000,3330.000,3330.0,0.000000,0.000000,150800,2334312600,0.000065
...,...,...,...,...,...,...,...,...,...,...
2332526,2021-12-03,9990,563.794118,532.500,528.0,-0.067792,-0.008523,44200,1238819500,0.000036
2332527,2021-12-03,9991,831.617647,787.875,794.0,-0.047377,0.007714,35900,1238819500,0.000029
2332528,2021-12-03,9993,1708.117647,1680.625,1645.0,-0.038369,-0.021657,7200,1238819500,0.000006
2332529,2021-12-03,9994,2404.852941,2370.125,2389.0,-0.006636,0.007901,6500,1238819500,0.000005
